In [1]:
import graspy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import dask
from dask.distributed import Client, progress
import dask.dataframe as ddf
#uncomment when placed into model folder not doc
#from .base import BaseGraphEstimator, _calculate_p

from scipy import stats
from scipy.special import expit
import pandas as pd 
import mizani as miz

from graspy.simulations import sbm, er_np, er_nm, siem
from graspy.plot import heatmap
from graspy.models import SBMEstimator

import statsmodels.api as sm
import statsmodels.formula.api as smf

from graspy.models.base import BaseGraphEstimator 
from graspy.utils.utils import (
    augment_diagonal,
    cartprod,
    import_graph,
    is_unweighted,
    remove_loops,
    symmetrize,
)
import sys
from dfply import *
import multiprocessing as multiproc

def gen_pmat(num_blocks, effect_size=0):
    if num_blocks == 2:
        # tend to [[a, b], [b, a]]
        pmat = np.array([[.5 + effect_size/2, .5 - effect_size/2], [.5 - effect_size/2, .5 + effect_size/2]])
    elif num_blocks == 3:
        # tend to [[a, b], [b, d]]
        pmat = np.array([[.5 + effect_size/2, .5], [.5, .5 - effect_size/2]])
    elif num_blocks == 4:
        # tend to [[a, b], [c, d]]
        pmat = np.array([[.5 + effect_size/2, .5 + effect_size/4], [.5 - effect_size/4, .5 - effect_size/2]])
    return pmat

def quad_sbm(n, pmat, weighted=False, return_labels=False):
    comm = [n//2, n//2]
    if weighted:
        raise NotImplementedError()
    else:
        mat = graspy.simulations.sbm(comm, pmat, directed=True, loops=True, return_labels=return_labels)
    return mat
        
def sample_quad_sbm(n, effect_size=0, num_blocks=4, weighted=False, return_labels=False):
    pmat = gen_pmat(num_blocks, effect_size)
    mat = quad_sbm(n, pmat, weighted=weighted, return_labels=return_labels)
    return mat

def sbm_2(n, effect_size=0, weighted=False, return_labels=True):
    return sample_quad_sbm(n, effect_size=effect_size, num_blocks=2, weighted=weighted, return_labels=return_labels)

def sbm_3(n, effect_size=0, weighted=False, return_labels=True):
    return sample_quad_sbm(n, effect_size=effect_size, num_blocks=3, weighted=weighted, return_labels=return_labels)

def sbm_4(n, effect_size=0, weighted=False, return_labels=True):
    return sample_quad_sbm(n, effect_size=effect_size, num_blocks=4, weighted=weighted, return_labels=return_labels)

vals = [(0.05, 0), (0.2, 1), (0.1, 2)]
vals.sort()
print(vals)

# the color for each test statistic
test_stat_cols = {'FET': 'red',
                  'LRT': 'teal',
                  'KW': 'blue',
                  'DCorr': 'green',
                  'ANOVA': 'orange',
                  'Chi2': 'purple'}

# an array tracking the test statistics themselves
test_stats = {'FET': 'fisher_exact',
            'Chi2': 'chi2',
            'LRT': 'lrt',
            'DCorr': 'dcorr',
            'KW': 'kw',
            'ANOVA': 'anova'}

# an array for when to use different test statistics
test_stat_use = {'FET': ['Unweighted'],
                 'LRT': ['Unweighted'],
                 'Chi2': ['Unweighted'],
                 'DCorr': ["Unweighted", "Weighted"],
                 'KW': ["Unweighted", "Weighted"],
                 'ANOVA': ["Unweighted", "Weighted"]}

simulations = {
    'Unweighted': {'abba': {'fn': sbm_2, 'eff_sz': np.linspace(0, .15, 10),'kwarg': {"weighted": False}},
                   'abbd': {'fn': sbm_3, 'eff_sz': np.linspace(0, .15, 10),'kwarg': {"weighted": False}},
                   'abcd': {'fn': sbm_4, 'eff_sz': np.linspace(0, .15, 10),'kwarg': {"weighted": False}}
                  }
}

nrep = 100
ncores = multiproc.cpu_count()-2
nvertices = np.round(np.linspace(20, 100, 10))

client = Client(threads_per_worker=1, n_workers=ncores)
client

def run_exp(row):
    # generate simulation using the dictionary defined above
    sim_dict = simulations[row["Weighting"]][row["Simulation"]]
    if sim_dict["kwarg"] is not None:
        (A, C) = sim_dict["fn"](row["n"], row["Effect_Size"], **sim_dict["kwarg"])
    else:
        (A, C) = sim_dict["fn"](row["n"], row["Effect_Size"])
        
    # run sim on all test statistics
    p_vals = []
    models = []
    stat_names = []
    for test_name, test in test_stats.items():
        # check if statistic is appropriate for the setting
        if row["Weighting"] in test_stat_use[test_name]:
            try:
                e = SBMEstimator()
                pval, model = e.estimate_block_structure(A, C, ["abba", "abbd", "abcd"], test_method=test)
                p_vals.append(pval)
                models.append(model)
                stat_names.append(test_name)
            except:
                ex_str = "Failed Test: {}, Simulation: {}, n: {}, Effect Size: {}"
                print(ex_str.format(test_name, row["Simulation"], row["n"], row["Effect_Size"]))
                p_vals.append(float("NaN"))
                models.append("")
        else:
            p_vals.append(float("NaN"))
            models.append("")
    return tuple([row["Simulation"], row["Weighting"], row["n"],
                 row["Effect_Size"], row["i"], *p_vals, *models])

exps = []
for weighting, sims in simulations.items():
    for sim_name, sim in sims.items():
        for es in sim["eff_sz"]:
            for n in nvertices:
                for i in range(nrep):
                    exps.append([sim_name, weighting, n, es, i])
sim_exps = pd.DataFrame(exps, columns=["Simulation", "Weighting", "n", "Effect_Size", "i"])
sim_exps["n"] = sim_exps["n"].astype(int)
print(sim_exps.head())
print(sim_exps.shape)

sim_exps = ddf.from_pandas(sim_exps, npartitions=ncores*1.5)
sim_results = sim_exps.apply(lambda x: run_exp(x), axis=1, result_type='expand',
                             meta={0: str, 1: str, 2: int, 3: float, 4: int,
                                   5: float, 6: float, 7: float, 8:float, 9:float, 10:float,
                                   11: str, 12: str, 13: str, 14: str, 15: str, 16: str})
sim_results

